In [28]:
import pandas as pd
from jinja2 import Template, Environment, FileSystemLoader
import os
import subprocess

In [5]:
fn_pubs = '../markdown_generator/publications.tsv'
df_pubs = pd.read_csv(fn_pubs, sep='\t')
pubs = df_pubs.to_dict(orient='records')

### Generate Markdown from Template

In [41]:
# Load the Markdown template
with open('template.md') as file:
    template_content = file.read()

# Create a Jinja2 Template object
template = Template(template_content)

# Render the template with the publications data
markdown_output = template.render(publications=pubs)

# Save the rendered Markdown content to a file
with open('cv.md', 'w') as file:
    file.write(markdown_output)


### Try to use custom template for html

In [42]:
# Read the Markdown content from file
with open('cv.md') as file:
    markdown_content = file.read()

# Create a Jinja2 environment and load the template file
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template('cv_template.html')

# Render the template with the Markdown content
rendered_content = template.render(content=markdown_content)

# Write the rendered content to a temporary Markdown file
with open('temp.md', 'w') as file:
    file.write(rendered_content)

# Convert the temporary Markdown file to HTML using Pandoc
subprocess.run(['pandoc', 'temp.md', '-o', 'cv.html'])

# Remove the temporary Markdown file
subprocess.run(['rm', 'temp.md'])


CompletedProcess(args=['rm', 'temp.md'], returncode=0)

### Change to use bibtex (not csv)

In [83]:
import bibtexparser

##### parse bibtex file

In [84]:
# Read the BibTeX file using bibtexparser
with open('publications.bib') as file:
    publications = bibtexparser.load(file).entries

with open('talks.bib') as file:
    talks = bibtexparser.load(file).entries


##### add publications / talks using jinja2

In [85]:
# Load the Markdown template
with open('template.md') as file:
    template_content = file.read()

# Create a Jinja2 Template object
template = Template(template_content)

# Render the template with the publications data
markdown_output = template.render(publications=publications, talks=talks)

# Save the rendered Markdown content to a file
with open('cv.md', 'w') as file:
    file.write(markdown_output)


##### bold every instance of me name

In [86]:
import re

def bold_string_in_markdown(markdown_content, target_string):
    markdown_new = markdown_content.replace(target_string, '**' + target_string + '**')

    return markdown_new

# Read the Markdown content from file
with open('cv.md') as file:
    markdown_content = file.read()

# Bold the target string in the Markdown content
bolded_content = bold_string_in_markdown(markdown_content, 'Ragland, John')

# Save the bolded Markdown content to a new file
with open('cv.md', 'w') as file:
    file.write(bolded_content)


##### build html file from custom template cv_template.html

In [87]:
# Read the Markdown content from file
with open('cv.md') as file:
    markdown_content = file.read()

# Create a Jinja2 environment and load the template file
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template('cv_template.html')

# Render the template with the Markdown content
rendered_content = template.render(content=markdown_content)

# Write the rendered content to a temporary Markdown file
with open('temp.md', 'w') as file:
    file.write(rendered_content)

# Convert the temporary Markdown file to HTML using Pandoc
subprocess.run(['pandoc', 'temp.md', '-o', 'cv.html'])

# Remove the temporary Markdown file
subprocess.run(['rm', 'temp.md'])


CompletedProcess(args=['rm', 'temp.md'], returncode=0)

##### Build PDF from Markdown

In [90]:
# Read the Markdown content from file
with open('cv.md') as file:
    markdown_content = file.read()

# Create a Jinja2 environment and load the template file
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template('cv_template.tex')

# Render the template with the Markdown content
rendered_content = template.render(content=markdown_content)

# Write the rendered content to a temporary Markdown file
with open('temp.md', 'w') as file:
    file.write(rendered_content)

# Convert the temporary Markdown file to HTML using Pandoc
subprocess.run(['pandoc', 'temp.md', '-o', 'cv.pdf'])

# Remove the temporary Markdown file
subprocess.run(['rm', 'temp.md'])


Error producing PDF.
! LaTeX Error: Can be used only in preamble.

See the LaTeX manual or LaTeX Companion for explanation.
Type  H <return>  for immediate help.
 ...                                              
                                                  
l.55 \documentclass



CompletedProcess(args=['rm', 'temp.md'], returncode=0)

## Create code that can pull from zotero

In [8]:
import pyzotero
import bibtexparser
from bibtexparser.bwriter import BibTexWriter

from pyzotero import zotero

In [11]:

# Configure Zotero API access
library_id = '5442436'
library_type = 'user'
api_key = 'B7alYThzKtI8l30sMofxru2f'

# Create a Zotero API client
zot = zotero.Zotero(library_id, library_type, api_key)


In [23]:
collection_name = 'talks'

for collection in zot.collections():
    if collection['data']['name'] == 'talks':
        talks_key = collection['key']
    if collection['data']['name'] == 'papers':
        papers_key = collection['key']
    if collection['data']['name'] == 'thesi':
        thesi_key = collection['key']


In [26]:
items = zot.collection_items(talks_key)

In [29]:
item_details = zot.item(items[0]['key'])

In [30]:
item_details

{'key': 'D6BKAJJX',
 'version': 4735,
 'library': {'type': 'user',
  'id': 5442436,
  'name': 'John Ragland',
  'links': {'alternate': {'href': 'https://www.zotero.org/john_ragland',
    'type': 'text/html'}}},
 'links': {'self': {'href': 'https://api.zotero.org/users/5442436/items/D6BKAJJX',
   'type': 'application/json'},
  'alternate': {'href': 'https://www.zotero.org/john_ragland/items/D6BKAJJX',
   'type': 'text/html'}},
 'meta': {'creatorSummary': 'Douglass et al.',
  'parsedDate': '2023-03-01',
  'numChildren': 0},
 'data': {'key': 'D6BKAJJX',
  'version': 4735,
  'itemType': 'journalArticle',
  'title': 'Overview of distributed acoustic sensing technology and recently acquired data sets',
  'creators': [{'creatorType': 'author',
    'firstName': 'Alexander S.',
    'lastName': 'Douglass'},
   {'creatorType': 'author', 'firstName': 'John', 'lastName': 'Ragland'},
   {'creatorType': 'author', 'firstName': 'Shima', 'lastName': 'Abadi'}],
  'abstractNote': 'Fiber optic distributed 